# 摘要

特征组合对于许多模型预测都是非常重要的。FM模型将特征交叉权重用embedding内积进行估计，并且可以对没有见过的组合特征进行泛化。DNN模型可以学习低阶与高阶的特征组合，但简单的DNN模型对于特征组合的生成方式是隐式的、bit-wise level的。
本篇论文提出一个全新的压缩交互网络（Compressed Interaction Network），可以显示地、vector-wise level地进行交叉特征生成，CIN与CNN和RNN共享了一些功能。
除此之外，本文还将CIN与DNN模型结合形成统一的模型，即eXtreme Deep Factorization Machine（xDeepFM）。
一方面，xDeepFM可以显示地学习特征bounded-degree（有限阶）的特征交叉；另一方面，可以学习隐式地学习任意低阶与高阶特征交叉。

# 简介

特征叉乘（特征交叉）对于模型预测具有非常重要的作用。但传统的特征交叉有三种主要的缺点：
- 获得高质量的特征往往投入的成本也越大
- 在大规模数据和系统中，手动提取所有的交叉特征往往是不可行的
- hand-crafted交叉特征对于训练集中没有出现的交叉信息无法进行泛化

现有的一些工作回顾：
- FM只能捕捉二阶特征；尽管FM可以扩展到高阶组合，但FM倾向于对于所有特征进行交叉（包括useful和useless特征），useless特征往往会给模型带来噪声
- FNN和PNN都focus在高阶特征交互方面，对于low-order的交互特征考虑较少
- W&D与DeepFM通过引入shallow和deep组合的hybrid模型结构，使得模型可以学习memorization和generalization，兼顾了低阶和高阶特征组合的学习

目前的工作中，基本都通过引入DNN来对高阶特征交叉进行建模。然而DNN对于特征交叉的最大阶并没有一个理论证明的结果；除此之外，DNN是bit-wise level（对应FM是vector-wise level）且是隐式学习交互特征。

因此本文提出基于NN的显式地、vector-wise level的模型。xDeepFM的工作是基于DCN的，目标都是去有效地capture有限阶的特征交叉。

主要贡献：
- 提出xDeepFM模型，可以同时学习显式与隐式的高阶特征交叉，并且不需要人工特征工程
- 提出CIN网络，可以显示地学习高阶特征交叉，且是vector-wise level
- xDeepFM在公开数据集上表现最好

# Implicit and Explicit

## Implicit High-order Interactions

NN中的全连接层的高阶组合是bit-wise的方式，换句话说，即对于同一个feature的embedding vector，其都是互相影响的。典型的如W&D、FNN、DCN中的Deep部分，都采用的是implicit bit-wise方式。

PNN与DeepFM有些许不同，除了DNN部分外，这两个模型都增加二阶交叉组合。因此包含了bit-wise和vector-wise。

![](pictures/xdeepfm-implicit.png)

## Explicit High-order Interactions

DCN中Cross Network的提出就是为了显式地对high-order feature interactions进行建模。Cross Network中主要采用显示交叉：$x_{l+1}=x_0 x_l^T w_{l+1} + b_{l+1} + x_l$

DCN学习了一种特殊的高阶特征交互类型，其中CrossNet中每个隐藏层都是输入$x_0$的标量倍（注意，这里$x_k$与$x_0$并不是线性关系）。

![](pictures/xdeepfm-proof.png)

DCN可以高效地显式学习高阶交叉特征，但仍有以下不足：
- CrossNet的输出受到特殊形式的限制，每个隐藏层都是x0的标量倍数
- 交互方式仍然是bit-wise的方式

![](pictures/xdeepfm-explicit.png)

# xDeepFM

## Compressed Interaction Network

本文基于以上分析提出了一个新的网络结构，称为压缩交互网络（CIN），网络的提出主要基于一下考虑：
- 交互的应用是vector-wise的，而不是bit-wise level
- 高阶特征交互应该是explicitly
- 网络复杂度不应该随着交互阶的增加而呈指数增长

CIN的Cross Layer的公式如下：

![](pictures/cin-formulation.png)

> 其中哈达玛积是指：$\langle a_1, a_2, a_3 \rangle \circ \langle b_1, b_2, b_3 \rangle = \langle a_1 b_1, a_2 b_2, a_3 b_3\rangle$

在CIN的理解上，单看公式是比较复杂的，原论文总将其分成了两步来理解：
- 使用外积的方式将embedding在D维度上展开，将两个2维的feature map转化为三维的tensor
- 再使用卷积的方式对三维tensor进行压缩（所以被叫做compressed net）

![](pictures/cin-compressed.png)

上图展示了CIN的计算过程：
- (a)图展示了两个feature map在D维上进行交叉，具体来说，输入是两个feature map：

$$f1\in\mathbb{R}^{h\times D}$$

$$f2\in\mathbb{R}^{m\times D}$$

经过计算会得到一个三维tensor：

$$z\in \mathbb{R}^{h\times m \times D}$$

- (b)通过卷积方式将高度为D的tensor进行压缩

通过上述方式可以串行地得到显式地交叉feature map，最后通过sum pooling的方式将D维进行压缩后concat得到最终的向量，进而得到output layer。

![](pictures/cin.png)

## CIN Analysis

### Space Complexity

对于每个CIN层，其压缩后的$h$个feature map，与3d的tensor的每一层都是全连接的，而这个全连接的参数共享的，所以有$H_{k-1}$\times $m$个参数。那么对于第$k$层，共有$H_k\times H_{k-1}\times m$个参数。

对于CIN的最后全连接层，即sum-pooling以后到output的fc层，共有$\sum_{k=1}^T H_k$个参数。

因此整个CIN部分的参数个数为：

$$\sum_{k=1}^T (H_k\times H_{k-1}\times m)+H_k$$

可以发现CIN的参数规模与embedding维度是无关的。而对于T层的DNN来说，参数规模为$m\times D\times H_1 + H_T + \sum_{k=2}^T H_k\times H_{k-1}$，整个参数规模会随着$D$的增加而增加。

### Time Complexity

计算$Z_{k+1}$的时间复杂度为$O(mHD)$。因为在每一层中都有$H$个feature map，共有$T$层CIN Layer的话，时间复杂度为$(mH^2DT)$。

对于普通DNN来说，时间复杂度为$O(mHD+H^2T)$。相比来说CIN的不足就是时间复杂度太高。

## Commbination with Implicit Networks

DNN的Implicit和CIN的explicit可以互补，因此将两者结合是很自然的方式。xDeepFM的结构与W&D和DeepFM和很相似，都是与DNN相结合。

xDeepFMM既包含了low-order和high-order feature的组合，同时也包含了implicit和explicit的特征交叉：

![](pictures/xdeepfm-formula.png)

![](pictures/xdeepfm-overview.png)

### 与FM与DeepFM的关系

- 当CIN的depth和feature map均为1时，xDeepFM就是DeepFM的一种推广，在DeepFM的基础上增加了对于FM层输出的linear regression
- 当删掉DNN部分后，同时将使用sum filter，将$Z$的每个D层直接进行sum，此时xDeepFM就退化为传统的FM

# Experiments

作者使用一下两个指标在3个公开dataset上进行试验：
- AUC：衡量预估结果的排序性
- LogLoss：衡量预估的分数与真实标签之间的距离

CIN的超参实验：

![](pictures/cin-experiment.png)

结论：
- Layer适中就好，太多会过拟合
- 结点个数100个左右合适
- 恒等函数（即没有激活函数）时效果最好

# xDeepFM的不足

- Sum pooling部分会损失信息，可以借鉴Attention思想
- CIN部分的时间复杂度过高